In [1]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm 
import os
from tqdm import tqdm 
import pickle

model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

model = tensorflow.keras.Sequential([    
    model,
    GlobalMaxPooling2D()
])

def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224)) 
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0) 
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten() 
    normalized_result = result / norm(result)
    
    return normalized_result

filenames = []
for file in os.listdir('images'):
    filenames.append(os.path.join('images',file))


feature_list = []
for file in tqdm(filenames):
    feature_list.append(extract_features(file,model)) 

pickle.dump(feature_list,open('embeddings.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))

100%|██████████| 2228/2228 [08:37<00:00,  4.31it/s]


In [2]:
import pickle
import tensorflow
import numpy as np
from numpy.linalg import norm
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from sklearn.neighbors import NearestNeighbors
import cv2
# loading the files created above with features and filename of images stored
feature_list = np.array(pickle.load(open('embeddings.pkl','rb')))
filenames = pickle.load(open('filenames.pkl','rb'))
# model loading again for the user input image
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])
#sample contains an image of every type in the dataset so that we get the vector through reset of that image. Limitation: input must be from the sample
def switch(val):
    return{
    "jersey": image.load_img('sample/jersey.jpg',target_size=(224,224)),
        
    "watch": image.load_img('sample/watch.jpg',target_size=(224,224)),
        
    "trouser": image.load_img('sample/trouser.jpg',target_size=(224,224)),
        
    "short": image.load_img('sample/short.jpg',target_size=(224,224)),
        
    "top": image.load_img('sample/top.jpg',target_size=(224,224)),
        
    "skirt": image.load_img('sample/skirt.jpg',target_size=(224,224)),
        
       "shoe": image.load_img('sample/shoe.jpg',target_size=(224,224)),
        
    "sock": image.load_img('sample/sock.jpg',target_size=(224,224)),
    }.get(val,"nothing")

val = str(input("Enter your search: "))
img = switch(val)
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessed_img).flatten()
normalized_result = result / norm(result)

neighbors = NearestNeighbors(n_neighbors=6,algorithm='brute',metric='euclidean')
neighbors.fit(feature_list)

distances,indices = neighbors.kneighbors([normalized_result]) 


for file in indices[0][1:6]: 
    temp_img = cv2.imread(filenames[file]) 
    cv2.imshow('output',cv2.resize(temp_img,(512,512)))
    cv2.waitKey(0)

Enter your search: shoe
